# Bölüm 4: Veri ve Talimatları Ayırma

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [1]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [
          "Merhaba, nasılsın?",
        ],
      },
      {
        "role": "model",
        "parts": [
          "Merhaba! Ben bir dil modeliyim, bu yüzden hislerim yok. \n\nSen nasılsın?  \n",
        ],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

C:\Users\AKE\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## Ders

Çoğu zaman, tam promptlar yazmak istemeyiz, bunun yerine Gemini'a göndermeden önce **ek girdiler ile daha sonra değiştirebileceğimiz prompt şablonları** isteriz. Gemini'ın her seferinde aynı şeyi yapmasını istiyorsanız bu yöntem kullanışlı olabilir, ancak Gemini'ın bu tarz görevler için kullandığı bu ek veriler her seferinde farklı olabilir. 

Neyse ki, bunu **komut isteminin sabit iskeletini değişken kullanıcı girdisinden ayırarak, ardından komut isteminin tamamını Gemini'a göndermeden önce kullanıcı girdisini komut isteminde değiştirerek** oldukça kolay bir şekilde yapabiliriz. 

Aşağıda, değiştirilebilir bir komut istemi şablonunun nasıl yazılacağını ve kullanıcı girdisinin nasıl değiştirileceğini adım adım anlatacağız.

### Örnekler

Bu ilk örnekte, Gemini'dan bir hayvan sesi üreteci olarak hareket etmesini istiyoruz. Gemini'a gönderilen tam komut promptuna sadece girdi (bu durumda İnek) ile değiştirilmiş `PROMPT_TEMPLATE` olduğuna dikkat edin. Komut promptunun tamamını yazdırdığımızda “İnek” kelimesinin bir f dizesi (f-string) aracılığıyla `ANIMAL` yer tutucusunun (placeholder) yerine geçtiğine dikkat edin.

**Not:** Placeholder değişkeninize pratikte özel bir isim vermek zorunda değilsiniz. Bu örnekte `ANIMAL` olarak adlandırdık, ancak kolayca `CREATURE` veya `A` olarak da adlandırabilirdik (ancak genellikle değişken adlarınızın belirli ve alakalı olması iyidir, böylece prompt şablonunuzun ikame olmadan bile anlaşılması kolaydır, sadece kullanıcı ayrıştırılabilirliği için). Değişkeninize verdiğiniz adın, prompt şablonu f-string için kullandığınız ad olduğundan emin olun.

In [4]:
# Variable content
ANIMAL = "İnek"

# Prompt template with a placeholder for the variable content
PROMPT = f"Sana bir hayvanın adını söyleyeceğim. Lütfen o hayvanın çıkardığı sesle cevap ver. {ANIMAL}"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Sana bir hayvanın adını söyleyeceğim. Lütfen o hayvanın çıkardığı sesle cevap ver. İnek

------------------------------------- Gemini's response -------------------------------------
Mööö! 🐄 



Neden girdileri bu şekilde ayırmak ve değiştirmek isteyelim? **Prompt şablonları tekrar eden görevleri basitleştirmeye yarar**. Diyelim ki üçüncü taraf kullanıcıları komut promptuna içerik göndermeye davet eden bir komut prompt yapısı oluşturdunuz (bu durumda isterler, sesini oluşturmak istedikleri hayvan türü oluyor). Bu üçüncü taraf kullanıcıların bilgi promptunun tamamını yazması ya da görmesi bile gerekmemekte. Tek yapmaları gereken değişkenleri doldurmak.

Biz burada değişkenleri ve f-stringleri kullanarak bu yerleştirmeyi yapıyoruz, ancak siz bunu format() yöntemiyle de yapabilirsiniz.

**Not:** Prompt şablonları istenildiği kadar değişkene sahip olabilir!

Bu gibi yerleştirme değişkenlerini tanıtırken, Gemini'ın **değişkenlerin nerede başlayıp nerede bittiğini (talimatlar veya görev tanımlarına kıyasla) bildiğinden** emin olmak çok önemlidir. Talimatlar ile yerleştirilen değişken arasında hiçbir ayrımın olmadığı bir örneğe bakalım.

İnsan gözüyle bakıldığında, aşağıdaki komut şablonunda değişkenin nerede başlayıp nerede bittiği çok açıktır. Ancak, tamamen ikame edilmiş promptta, bu sınırlama belirsiz hale gelir.

In [3]:
# Variable content
EMAIL = "Yarın sabah 6'da gel çünkü ben CEO'yum ve böyle olmasını istiyorum."

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Gemini. {EMAIL} <----- Bu e-postayı daha kibar hale getir ancak başka hiçbir şeyi değiştirme."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Hey Gemini. Yarın sabah 6'da gel çünkü ben CEO'yum ve böyle olmasını istiyorum. <----- Bu e-postayı daha kibar hale getir ancak başka hiçbir şeyi değiştirme.

------------------------------------- Gemini's response -------------------------------------
Merhaba Gemini,

Yarın sabah saat 6'da gelmeni rica ediyorum. CEO olarak, varlığının o saatte gerekli olduğunu düşünüyorum. 

Teşekkürler,
[Adınız] 



Burada, **Gemini “Hey Gemini” ifadesinin yeniden yazması gereken e-postanın bir parçası olduğunu düşünüyor!** Bunu anlayabilirsiniz çünkü yeniden yazmaya "Merhaba Gemini" ile başlıyor. İnsan gözüyle bakıldığında, özellikle bilgi prompt şablonunda e-postanın nerede başlayıp nerede bittiği açıktır, ancak değiştirildikten sonraki bilgi promptunda bu çok daha az net hale gelir.

Peki bunu nasıl çözeceğiz? **Girdiyi XML etiketleri içine alarak**! Bunu aşağıda yaptık ve görebileceğiniz gibi, çıktıda artık Merhaba Gemini” yok.

[XML etiketleri] (https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/structure-prompts) `<tag><tag>` gibi açılı ayraç etiketleridir. Çiftler halinde gelirler ve `<tag>` gibi bir açılış etiketinden ve `<tag>` gibi `/` ile işaretlenmiş bir kapanış etiketinden oluşurlar. XML etiketleri, aşağıdaki gibi içeriğin etrafını sarmak için kullanılır: `<tag>content<tag>`.

**Not:** Gemini, çok çeşitli ayırıcıları ve sınırlayıcıları tanıyabilir ve bunlarla çalışabilir. Özellikle XML etiketlerini bir prompt düzenleme mekanizması olarak tanımak üzere eğitildiğinden, Gemini için ayırıcı olarak **özellikle XML etiketlerini kullanmanızı** öneririz. İşlev çağırma dışında, **Gemini'ın performansınızı en üst düzeye çıkarmak için kullanmanız için eğitildiği özel bir sos XML etiketi yoktur**. Gemini'ı bilerek bu şekilde çok yumuşak ve özelleştirilebilir hale getirdik.

In [7]:
# Variable content
EMAIL = "Yarın sabah 6'da gel çünkü ben CEO'yum ve böyle olmasını istiyorum."

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Gemini. <email>{EMAIL}<email> <----- Bu e-postayı daha kibar hale getir ancak başka hiçbir şeyi değiştirme."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Hey Gemini. <email>Yarın sabah 6'da gel çünkü ben CEO'yum ve böyle olmasını istiyorum.<email> <----- Bu e-postayı daha kibar hale getir ancak başka hiçbir şeyi değiştirme.

------------------------------------- Gemini's response -------------------------------------
<email>Merhaba,

Yarın sabah 6'da ofiste olmanı rica ediyorum.

Teşekkürler,
[Adınız]
</email>


XML etiketlerinin bize nasıl yardımcı olabileceğine dair başka bir örnek görelim. 

Aşağıdaki komut isteminde, **Gemini komut promptunun hangi kısmının talimat, hangi kısmının girdi olduğunu yanlış yorumlamaktadır**. Kullanıcı (`SENTENCES` değişkenini dolduran kişi) muhtemelen bunu istemediği halde, biçimlendirme nedeniyle `Her biri tavşan gibi bir hayvan hakkında` ifadesini yanlışlıkla listenin bir parçası olarak değerlendirir.

In [8]:
# Variable content
SENTENCES = """- İneklerin sesini seviyorum.
- Bu cümle örümcekler hakkında
- Bu cümle köpeklerle ilgili gibi görünebilir ama aslında domuzlarla ilgilidir"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Aşağıda bir cümle listesi var. Bana listedeki ikinci maddeyi söyleyin.

- Her biri tavşan gibi bir hayvan hakkında.
{SENTENCES}"""

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Aşağıda bir cümle listesi var. Bana listedeki ikinci maddeyi söyleyin.

- Her biri tavşan gibi bir hayvan hakkında.
- İneklerin sesini seviyorum.
- Bu cümle örümcekler hakkında
- Bu cümle köpeklerle ilgili gibi görünebilir ama aslında domuzlarla ilgilidir

------------------------------------- Gemini's response -------------------------------------
Listedeki ikinci madde: **"İneklerin sesini seviyorum."** 



Bunu düzeltmek için tek yapmamız gereken **kullanıcı girdi cümlelerini XML etiketleriyle çevrelemek**. Bu, Gemini'a "Her biri tavşan gibi bir hayvan hakkında." cümlesinden önceki yanıltıcı kısa çizgiye rağmen girdi verilerinin nerede başlayıp nerede bittiğini gösterir.

In [9]:
# Variable content
SENTENCES = """- İneklerin sesini seviyorum.
- Bu cümle örümcekler hakkında
- Bu cümle köpeklerle ilgili gibi görünebilir ama aslında domuzlarla ilgilidir"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Aşağıda bir cümle listesi var. Bana listedeki ikinci maddeyi söyleyin.

- Her biri tavşan gibi bir hayvan hakkında.
<sentences>
{SENTENCES}
<sentences>"""

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Aşağıda bir cümle listesi var. Bana listedeki ikinci maddeyi söyleyin.

- Her biri tavşan gibi bir hayvan hakkında.
<sentences>
- İneklerin sesini seviyorum.
- Bu cümle örümcekler hakkında
- Bu cümle köpeklerle ilgili gibi görünebilir ama aslında domuzlarla ilgilidir
<sentences>

------------------------------------- Gemini's response -------------------------------------
Listedeki ikinci madde: 

**- Bu cümle örümcekler hakkında** 



**Not:** "Her biri bir hayvan hakkında" sorusunun yanlış versiyonunda, Gemini'ın bu örnek için istediğimiz şekilde yanlış yanıt vermesini sağlamak için kısa çizgiyi dahil etmek zorunda kaldık. Bu, yönlendirme konusunda önemli bir derstir: **Küçük ayrıntılar önemlidir**! Yazım hataları ve dilbilgisi hataları için yönlendirmelerinizi **her zaman gözden geçirmeye değer**. Gemini kalıplara karşı hassastır (ilk yıllarında, ince ayarlardan önce, ham bir metin tahmin aracıydı) ve siz hata yaptığınızda hata yapma olasılığı daha yüksektir, siz akıllıca konuştuğunuzda daha akıllı, aptalca konuştuğunuzda daha aptal vb.


---

## Alıştırmalar
- [Alıştırma 4.1 - Haiku Konusu](#exercise-41---haiku-topic)
- [Alıştırma 4.2 - Yazım Hatalı Köpek Sorusu](#exercise-42---dog-question-with-typos)
- [Alıştırma 4.3 - Köpek Sorusu Part 2](#exercise-42---dog-question-part-2)

### Alıştırma 4.1 - Haiku Konusu
`PROMPT` şablonunu `TOPIC` adlı bir değişkeni alacak ve konu hakkında bir haiku çıktısı verecek şekilde değiştirin. Bu alıştırma sadece f-string ile değişken şablonlama yapısını anlamanızı test etmek içindir.

In [ ]:
# Variable content
TOPIC = "Domuzlar"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Gemini's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("domuzlar", text.lower()) and re.search("haiku", text.lower()))

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### Alıştırma 4.2 - Yazım Hatalı Köpek Sorusu
Gemini'ın doğru yanıtı üretmesi için XML etiketleri ekleyerek `PROMPT`u düzeltin. 

Promptla ilgili başka hiçbir şeyi değiştirmemeye çalışın. Dağınık ve hatalarla dolu yazı kasıtlıdır, böylece Gemini'ın bu tür hatalara nasıl tepki verdiğini görebilirsiniz.

**NOT: Bu alıştırmaya özel olarak İngilizce metin değiştirilmemiş, orjinal metin kullanılmıştır.**

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Gemini's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### Alıştırma 4.3 - Köpek Sorusu Part 2
XML etiketleri **eklemeden** `PROMPT`u düzeltin. Bunun yerine, istemden sadece bir veya iki kelimeyi kaldırın.

Yukarıdaki alıştırmalarda olduğu gibi, istemle ilgili başka hiçbir şeyi değiştirmemeye çalışın. Bu size Gemini'ın ne tür bir dili ayrıştırabildiğini ve anlayabildiğini gösterecektir.

**NOT: Bu alıştırmaya özel olarak İngilizce metin değiştirilmemiş, orjinal metin kullanılmıştır.**

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Gemini's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Oyun Alanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Gemini. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Gemini. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))